# Quantization

## WHY?

float32의 연산이 int보다 복잡하다.  
float32의 구조가 부동소수점 구조인데  

| sign | exponential | mantissa |
| :--: | :---------: | :------: |
|  1   |      8      |    32    |

연산이 int보다 같은 bit에서 대략 4배 느리다.  
수를 int 구조로 바꿔서 연산하는 방식을 채택.

근거 중 하나는 부동소수점 구조가 넓은 범위의 수를 표현하기 위해서 만들어져있는데 weight 값들이 생각보다 넓게 퍼져있지 않음.  
이를 이용하여 (min, max) or (mean, scale)의 값을 이용하여 균일한 간격으로 분포시킬 수 있음.

## In pytorch

자체 함수가 존재. 하지만 아직 cpu에서 연산한다. (원래 gpu에서도 가능은 함)

In [1]:
import torch

In [2]:
float_tensor = torch.randn(2,2,3)
float_tensor

tensor([[[ 0.1803, -1.7519, -0.2755],
         [ 1.0332, -0.0269, -1.3415]],

        [[ 0.5794, -0.2408,  1.4850],
         [-2.3509,  1.7678, -0.7412]]])

In [3]:
zero_point = 0 # 반드시 int 여야 함.
scale = 1/100
q_tensor = torch.quantize_per_tensor(float_tensor, scale=scale, zero_point=zero_point, dtype=torch.qint8)
q_tensor

tensor([[[ 0.1800, -1.2800, -0.2800],
         [ 1.0300, -0.0300, -1.2800]],

        [[ 0.5800, -0.2400,  1.2700],
         [-1.2800,  1.2700, -0.7400]]], size=(2, 2, 3), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.01, zero_point=0)

위 결과를 보면 scale = 1/100, zero_point = 0에서 가능한 값의 범위는 \[-1.28, 1.27\]이다.  
그 밖의 결과는 clip되는 효과.

만약 범위를 늘리고 싶다면 scale을 키우거나 qint bit 수를 키우면 된다. (e.g. scale = 1/50)

In [4]:
scale = 1/50
q_tensor_1 = torch.quantize_per_tensor(float_tensor, scale=scale, zero_point=zero_point, dtype=torch.qint8)
q_tensor_1


tensor([[[ 0.1800, -1.7600, -0.2800],
         [ 1.0400, -0.0200, -1.3400]],

        [[ 0.5800, -0.2400,  1.4800],
         [-2.3600,  1.7600, -0.7400]]], size=(2, 2, 3), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.02, zero_point=0)

In [10]:
torch.quantize_per_channel(float_tensor, axis=2, scales=torch.tensor([1e-1, 1e-2, 1e-3]), zero_points=torch.tensor([-1, 0, 1]), dtype=torch.qint8)

tensor([[[ 0.2000, -1.2800, -0.1290],
         [ 1.0000, -0.0300, -0.1290]],

        [[ 0.6000, -0.2400,  0.1260],
         [-2.4000,  1.2700, -0.1290]]], size=(2, 2, 3), dtype=torch.qint8,
       quantization_scheme=torch.per_channel_affine,
       scale=tensor([0.1000, 0.0100, 0.0010], dtype=torch.float64),
       zero_point=tensor([-1,  0,  1]), axis=2)

## `Int` to quantized int (`qint`, value)

In [12]:
int_tensor = torch.randint(0, 100, size=(10,), dtype=torch.uint8)
q = torch._make_per_tensor_quantized_tensor(int_tensor, 1e-2, 0)
q

tensor([0.2200, 0.5800, 0.3100, 0.3300, 0.4900, 0.9000, 0.2000, 0.9800, 0.2900,
        0.1600], size=(10,), dtype=torch.quint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.01, zero_point=0)

In [13]:
# 수정도 가능

print(q[3])
q[3] = 0.444
print(q[3])

tensor(0.3300, size=(), dtype=torch.quint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.01, zero_point=0)
tensor(0.4400, size=(), dtype=torch.quint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.01, zero_point=0)


In [14]:
q.is_quantized

True

In [15]:
q.size

<function Tensor.size>

In [16]:
q.int_repr()

tensor([22, 58, 31, 44, 49, 90, 20, 98, 29, 16], dtype=torch.uint8)

# Introduction to quantization on pytorch

https://pytorch.org/blog/introduction-to-quantization-on-pytorch/
https://pytorch.org/docs/stable/quantization.html

## 1. Dynamic Quantization

이미 학습된 모델 -> quantization이 필요한 부분만 quantization.  
저장된 Weight를 memory에 불러오는 것이 bottle-neck인 경우. 일정 Weight를 int로 저장하므로써 가볍게 만든다. (e.g. BERT의 linear weight, [BERT example](https://pytorch.org/tutorials/intermediate/dynamic_quantization_bert_tutorial.html))

## 2. Post-Training Static Quantization

이미 학습된 모델 -> 처음부터 끝까지 quantization.  
전체 모델의 inference 시간이 오래 걸리는 경우. 사용하면 Computational Cost를 아낄 수 있다.  
학습때는 float32로 학습하고 추론시간 단축을 위해서 quantization. (`fbgemm` for x86, `qnnpack` for ARM ...)

-  fuse_modules : Quantization을 한 경우 \[conv, bn\], \[conv, bn, relu\]등을 하나의 모듈처럼 계산할 수 있다. (Computational Cost 이득, but 제한되는 option)

Model을 준비한 다음 Calibration을 하기 위해서 input 값을 넣어준다. 이를 통해 activation 값들이 어디에서 주로 존재하는지 확인한 후 \(min, max\)등을 사용한다.  
마지막으로 이렇게 통과하여 activation 값들이 나온 모델을 quantization 한다.

## 3. Quantization Aware Training

학습 단계에서 quantization을 할 것을 알고 있는 상황. (정확도도 이득을 보려고 함.)  
중간 layer에서 `fq`\(fake quantization\)을 넣어줘서 어디부터 어디까지 quantization 하려고 하는지만 알려준다. _\(따로 추가적인 layer나 int로 학습하거나 하지는 않음.\)_  
=> 따로 calibration이 필요 없음.